In [ ]:
tez_path = '../input/tez-lib/'
effnet_path = '../input/efficientnet-pytorch/'
timm_path = '../input/timm-pytorch-image-models/pytorch-image-models-master/'

import sys
sys.path.append(tez_path)
sys.path.append(effnet_path)
sys.path.append(timm_path)

In [ ]:
import os
import albumentations
import pandas as pd
import numpy as np
import timm
import cv2

import tez
from tez.datasets import ImageDataset

import torch
import torch.nn as nn
from torch.nn import functional as F

from efficientnet_pytorch import EfficientNet
from albumentations.pytorch import ToTensorV2


In [ ]:
INPUT_PATH = "../input/ranzcr-clip-catheter-line-classification/"
IMAGE_PATH = "../input/ranzcr-clip-catheter-line-classification/test/"
MODEL_PATH = "../input/ranzcr-model/"
IMAGE_SIZE = 600
label = 11

In [ ]:
df = pd.read_csv(os.path.join(INPUT_PATH, "sample_submission.csv"))

In [ ]:
class RanzcrModel1(tez.Model):
    def __init__(self):
        super().__init__()

        self.effnet = EfficientNet.from_name("efficientnet-b5")

        self.effnet._conv_stem.in_channels = 3
        weight = self.effnet._conv_stem.weight.mean(3, keepdim=True)
        self.effnet._conv_stem.weight = torch.nn.Parameter(weight)

        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(2048, 11)

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs, None, {}
    
class RanzcrModel2(tez.Model):
    def __init__(self):
        super().__init__()

        self.effnet = EfficientNet.from_name("efficientnet-b4")

        self.effnet._conv_stem.in_channels = 3
        weight = self.effnet._conv_stem.weight.mean(3, keepdim=True)
        self.effnet._conv_stem.weight = torch.nn.Parameter(weight)

        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(1792, 11)

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs, None, {}
    
class RanzcrModel3(tez.Model):
    def __init__(self,model_name='resnet200d', pretrained=False, num_classes=11):
        super().__init__()

        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, num_classes)
        self.criterion = nn.BCEWithLogitsLoss()
        
    def forward(self, image, targets=None):
        bs = image.size(0)
        features = self.model(image)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        #if targets != None:
        #    loss = self.criterion(output, targets)
        #    metrics = self.monitor_metrics(output, targets)
        #    return output, loss, metrics
        return output, None, None

In [ ]:
test_aug = albumentations.Compose(
    [
        albumentations.Resize(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.Transpose(p=0.5),
        albumentations.ShiftScaleRotate(p=0.5),
       albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.7),
       albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7),
       albumentations.CLAHE(clip_limit=(1,4), p=0.5),
       albumentations.OneOf([
           albumentations.OpticalDistortion(distort_limit=1.0),
           albumentations.GridDistortion(num_steps=5, distort_limit=1.),
           albumentations.ElasticTransform(alpha=3),
       ], p=0.2),
        albumentations.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
        ),
        ToTensorV2(),
    ],
    p=1.0,
)

In [ ]:
test_image_paths = "../input/ranzcr-clip-catheter-line-classification/test"

In [ ]:

'''
model1 = RanzcrModel1()
model1.load(os.path.join(MODEL_PATH, "efficentnet-b5_fold0.bin"))

model2 = RanzcrModel1()
model2.load(os.path.join(MODEL_PATH, "efficentnet-b5_fold1.bin"))

model3 = RanzcrModel1()
model3.load(os.path.join(MODEL_PATH, "efficentnet-b5_fold2.bin"))

model4 = RanzcrModel1()
model4.load(os.path.join(MODEL_PATH, "efficentnet-b5_fold3.bin"))

model5 = RanzcrModel1()
model5.load(os.path.join(MODEL_PATH, "efficentnet-b5_fold4.bin"))

model6 = RanzcrModel2()
model6.load(os.path.join(MODEL_PATH, "efficientnet-b4_fold0.bin"))

model7 = RanzcrModel2()
model7.load(os.path.join(MODEL_PATH, "efficientnet-b4_fold1.bin"))

model8 = RanzcrModel2()
model8.load(os.path.join(MODEL_PATH, "efficientnet-b4_fold2.bin"))

model9 = RanzcrModel2()
model9.load(os.path.join(MODEL_PATH, "efficientnet-b4_fold3.bin"))

model10 = RanzcrModel2()
model10.load(os.path.join(MODEL_PATH, "efficientnet-b4_fold4.bin"))


model11 = RanzcrModel3("resnet34d")
model11.load(os.path.join(MODEL_PATH, "resnet34d_fold0.bin"))

model12 = RanzcrModel3("resnet34d")
model12.load(os.path.join(MODEL_PATH, "resnet34d_fold1.bin"))

model13 = RanzcrModel3("resnet34d")
model13.load(os.path.join(MODEL_PATH, "resnet34d_fold2.bin"))

model14 = RanzcrModel3("resnet34d")
model14.load(os.path.join(MODEL_PATH, "resnet34d_fold3.bin"))

model15 = RanzcrModel3("resnet34d")
model15.load(os.path.join(MODEL_PATH, "resnet34d_fold4.bin"))

model16 = RanzcrModel3("resnet50d")
model16.load(os.path.join(MODEL_PATH, "resnet50d_fold0.bin"))

model17 = RanzcrModel3("resnet50d")
model17.load(os.path.join(MODEL_PATH, "resnet50d_fold1.bin"))

model18 = RanzcrModel3("resnet50d")
model18.load(os.path.join(MODEL_PATH, "resnet50d_fold2.bin"))

model19 = RanzcrModel3("resnet50d")
model19.load(os.path.join(MODEL_PATH, "resnet50d_fold3.bin"))

model20 = RanzcrModel3("resnet50d")
model20.load(os.path.join(MODEL_PATH, "resnet50d_fold4.bin"))
'''

model21 = RanzcrModel3()
model21.load(os.path.join(MODEL_PATH, "resnet_model_fold0.bin"))

model22 = RanzcrModel3()
model22.load(os.path.join(MODEL_PATH, "resnet_model_fold1.bin"))

model23 = RanzcrModel3()
model23.load(os.path.join(MODEL_PATH, "resnet_model_fold2.bin"))

model24 = RanzcrModel3()
model24.load(os.path.join(MODEL_PATH, "resnet_model_fold3.bin"))

model25 = RanzcrModel3()
model25.load(os.path.join(MODEL_PATH, "resnet_model_fold4.bin"))


In [ ]:
from torch.utils.data import DataLoader, Dataset

class TestDataset(Dataset):
    def __init__(self, df,image_path, transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.transform = transform
        self.image_path = image_path

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{self.image_path}/{file_name}.jpg'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        label = torch.tensor([[0]]).float()
        return {
                "image": image,
                "targets": label,
            }

In [ ]:
#Approach 1
final_preds = None
#model_list = [model1, model2, model3, model4, model5, model6, model7, model8, model9, model10, 
model_list  = [model21,model22, model23, model24, model25]
model_out = None
for model in model_list:
    for j in range(10):
        test_dataset = TestDataset(
            df,
            image_path=test_image_paths,
            transform=test_aug,
        )
        preds = model.predict(test_dataset, batch_size=6, n_jobs=-1)
        temp_preds = None
        for p in preds:
            if temp_preds is None:
                temp_preds = p
            else:
                temp_preds = np.vstack((temp_preds, p))
        if final_preds is None:
            final_preds = temp_preds
        else:
            final_preds += temp_preds
    final_preds /= 10
    if model_out is None:
        model_out = final_preds
    else:
        model_out += final_preds
        
model_out /= 5

In [ ]:
target_cols = df.columns[1:]

for i in range(model_out.shape[1]):
    df.loc[:, target_cols[i]] = model_out[:, i]

In [ ]:
df.to_csv('submission.csv', index=False)
df.head()